<a href="https://colab.research.google.com/github/Juliana001/AnaliseDeDados_projetosPessoais/blob/main/Projetos%20Machine%20Learning/IndicadorDePerfumes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Juliana](https://drive.google.com/uc?export=view&id=1PRtZkH3XKEHD7ROx0NvjS8SkubI-RKdu)

#O único objetivo desse notebook é me ajudar a comprar perfumes pela internet.

rerferência: https://www.kaggle.com/datasets/olgagmiufana1/fragrantica-com-fragrance-dataset

Passo a passo:
1- Analisar as fragrancias feminimas mais bem avaliadas
2- desenvolver um modelo que indique perfumes com base nos perfumes que você gosta
3- desenvolver um modelo que indique a não compra de perfumes com base nos perfumes que você não gosta

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dadosFragrancias = pd.read_csv('/content/DatasetFragrancias.csv')
dadosFragrancias_copia = dadosFragrancias.copy()
display(dadosFragrancias)

,url,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,Perfumer1,Perfumer2,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5
0,https://www.fragrantica.com/perfume/xerjoff/ac...,accento-overdose-pride-edition,xerjoff,Italy,unisex,"1,42",201,2022.0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",unknown,NaN,rose,woody,fruity,aromatic,floral
1,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2024,jean-paul-gaultier,France,women,"1,86",70,2024.0,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",unknown,NaN,citrus,white floral,sweet,fresh,musky
2,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2023,jean-paul-gaultier,France,unisex,"1,91",285,2023.0,"blood orange, yuzu","neroli, orange blossom","musk, white woods",natalie gracia-cetto,quentin bisch,citrus,white floral,sweet,fresh spicy,musky
3,https://www.fragrantica.com/perfume/bruno-bana...,pride-edition-man,bruno-banani,Germany,men,"1,92",59,2019.0,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",unknown,NaN,fruity,nutty,woody,tropical,NaN
4,https://www.fragrantica.com/perfume/jean-paul-...,le-male-pride-collector,jean-paul-gaultier,France,men,"1,93",632,2020.0,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",francis kurkdjian,NaN,aromatic,warm spicy,fresh spicy,cinnamon,vanilla
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24058,https://www.fragrantica.com/perfume/o-boticari...,floratta,o-boticario,Brazil,women,"4,81",31,1992.0,"ozonic notes, peach","lily-of-the-valley, jasmine","musk, amber",thierry lecoule,NaN,white floral,ozonic,fresh,musky,powdery
24059,https://www.fragrantica.com/perfume/sultan-pas...,cheval-d-arabie,sultan-pasha-attars,UK,unisex,"4,81",26,2020.0,"taif rose, indian oud, white rose, olibanum, e...","bulgarian rose, indian oud, hay, leather, cive...","white amber, leather, hay, elemi resin, kyara ...",sultan pasha,NaN,rose,amber,fresh spicy,animalic,aromatic
24060,https://www.fragrantica.com/perfume/darkbeat-p...,khaox,darkbeat-parfums,Spain,unisex,"4,85",26,2020.0,"mint, lime, rum","lily-of-the-valley, fig leaf, cardamom","musk, oakmoss, amber, patchouli",josé m. giraldo,NaN,green,aromatic,citrus,fresh spicy,woody
24061,https://www.fragrantica.com/perfume/parfumerie...,aoud-no-1,parfumerie-bruckner,Germany,unisex,"4,86",28,2011.0,"apple, peach, saffron","plum, orange blossom, jasmine","agarwood (oud), sandalwood, vanilla",unknown,NaN,fruity,sweet,oud,woody,powdery


In [ ]:
missingValues = dadosFragrancias.isnull().sum()
missingValues

,0
url,0
Perfume,0
Brand,0
Country,0
Gender,0
Rating Value,0
Rating Count,0
Year,2037
Top,0
Middle,0


In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

# =====================
# 1. Carregar dataset
# =====================
df = pd.read_csv("/content/DatasetFragrancias.csv")  # Substitua pelo caminho do seu CSV

# =====================
# 2. Criar coluna de listas de main accords
# =====================
df['mainaccords'] = df[['mainaccord1','mainaccord2','mainaccord3','mainaccord4','mainaccord5']].values.tolist()

# =====================
# 3. Limpar NaN e garantir strings
# =====================
def clean_mainaccords(acordes):
    return [str(a) for a in acordes if pd.notna(a)]

df['mainaccords'] = df['mainaccords'].apply(clean_mainaccords)

# =====================
# 4. Transformar em vetor binário (one-hot encoding)
# =====================
mlb = MultiLabelBinarizer()
X = mlb.fit_transform(df['mainaccords'])

# =====================
# 5. Função de recomendação
# =====================
def recomendar_perfumes(perfumes_usuario, top_n=100):
    # Filtrar apenas perfumes existentes no dataset
    perfumes_validos = []
    for perfume, marca in perfumes_usuario:
        if ((df['Perfume'] == perfume) & (df['Brand'] == marca)).any():
            perfumes_validos.append((perfume, marca))

    if len(perfumes_validos) == 0:
        return "Nenhum dos perfumes informados está no dataset."

    # Índices dos perfumes do usuário
    indices_usuario = df[
        df.apply(lambda row: (row['Perfume'], row['Brand']) in perfumes_validos, axis=1)
    ].index

    # Vetor médio do gosto do usuário
    vetor_usuario = X[indices_usuario].mean(axis=0).reshape(1, -1)

    # Similaridade com todos os perfumes
    similaridades = cosine_similarity(vetor_usuario, X).flatten()

    # Criar DataFrame com similaridade
    df_sim = df.copy()
    df_sim['similaridade'] = similaridades

    # Excluir perfumes que o usuário já indicou
    df_sim = df_sim[~df_sim.apply(lambda row: (row['Perfume'], row['Brand']) in perfumes_validos, axis=1)]

    # Ordenar por similaridade e Rating Value
    df_sim_sorted = df_sim.sort_values(by=['similaridade', 'Rating Value'], ascending=[False, False])

    # Retornar top_n perfumes
    return df_sim_sorted.head(top_n)[['Perfume', 'Brand', 'Rating Value', 'similaridade']]

# =====================
# 6. Interação com o usuário
# =====================
print("=== Sistema de Recomendação de Perfumes ===")
n = int(input("Quantos perfumes você quer informar que gosta? "))

perfumes_usuario = []
for i in range(n):
    nome_perfume = input(f"Digite o nome do perfume {i+1}: ")
    marca_perfume = input(f"Digite a marca do perfume {i+1}: ")
    perfumes_usuario.append((nome_perfume, marca_perfume))

top_n = 60

# =====================
# 7. Gerar recomendações
# =====================
recomendacoes = recomendar_perfumes(perfumes_usuario, top_n)
print("\n=== Recomendações ===")
print(recomendacoes.reset_index(drop=True))


=== Sistema de Recomendação de Perfumes ===
Quantos perfumes você quer informar que gosta? 3
Digite o nome do perfume 1: Liz
Digite a marca do perfume 1: O Boticario
Digite o nome do perfume 2: Cute
Digite a marca do perfume 2: La rive
Digite o nome do perfume 3: Cecita
Digite a marca do perfume 3: O Boticario

=== Recomendações ===


AttributeError: 'str' object has no attribute 'reset_index'

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

# =====================
# 1. Carregar dataset
# =====================
df = pd.read_csv("/content/DatasetFragrancias.csv")  # Substitua pelo caminho do seu CSV

# =====================
# 2. Criar coluna de listas de main accords
# =====================
df['mainaccords'] = df[['mainaccord1','mainaccord2','mainaccord3','mainaccord4','mainaccord5']].values.tolist()

# =====================
# 3. Limpar NaN e garantir strings
# =====================
def clean_mainaccords(acordes):
    return [str(a) for a in acordes if pd.notna(a)]

df['mainaccords'] = df['mainaccords'].apply(clean_mainaccords)

# =====================
# 4. Transformar em vetor binário (one-hot encoding)
# =====================
mlb = MultiLabelBinarizer()
X = mlb.fit_transform(df['mainaccords'])

# =====================
# 5. Função de recomendação
# =====================
def recomendar_perfumes(perfumes_usuario, top_n=60):
    # Filtrar apenas perfumes existentes no dataset
    perfumes_validos = []
    for perfume, marca in perfumes_usuario:
        if ((df['Perfume'] == perfume) & (df['Brand'] == marca)).any():
            perfumes_validos.append((perfume, marca))

    if len(perfumes_validos) == 0:
        return "Nenhum dos perfumes informados está no dataset."

    # Índices dos perfumes do usuário
    indices_usuario = df[
        df.apply(lambda row: (row['Perfume'], row['Brand']) in perfumes_validos, axis=1)
    ].index

    # Vetor médio do gosto do usuário
    vetor_usuario = X[indices_usuario].mean(axis=0).reshape(1, -1)

    # Similaridade com todos os perfumes
    similaridades = cosine_similarity(vetor_usuario, X).flatten()

    # Criar DataFrame com similaridade
    df_sim = df.copy()
    df_sim['similaridade'] = similaridades

    # Excluir perfumes que o usuário já indicou
    df_sim = df_sim[~df_sim.apply(lambda row: (row['Perfume'], row['Brand']) in perfumes_validos, axis=1)]

    # Filtrar apenas perfumes entre 2015 e 2020
    df_sim = df_sim[(df_sim['Year'] >= 2020) & (df_sim['Year'] <= 2025)]

    # Ordenar por similaridade e Rating Value
    df_sim_sorted = df_sim.sort_values(by=['similaridade', 'Rating Value'], ascending=[False, False])

    # Retornar top_n perfumes
    return df_sim_sorted.head(top_n)[['Perfume', 'Brand', 'Year', 'Rating Value', 'similaridade']]

# =====================
# 6. Interação com o usuário
# =====================
print("=== Sistema de Recomendação de Perfumes ===")
n = int(input("Quantos perfumes você quer informar que gosta? "))

perfumes_usuario = []
for i in range(n):
    nome_perfume = input(f"Digite o nome do perfume {i+1}: ")
    marca_perfume = input(f"Digite a marca do perfume {i+1}: ")
    perfumes_usuario.append((nome_perfume, marca_perfume))

top_n = 10

# =====================
# 7. Gerar recomendações
# =====================
recomendacoes = recomendar_perfumes(perfumes_usuario, top_n)
print("\n=== Recomendações (2020-2025) ===")
print(recomendacoes.reset_index(drop=True))


=== Sistema de Recomendação de Perfumes ===
Quantos perfumes você quer informar que gosta? 2
Digite o nome do perfume 1: liz
Digite a marca do perfume 1: o-boticario
Digite o nome do perfume 2: cecita
Digite a marca do perfume 2: o-boticario

=== Recomendações (2020-2025) ===
                                 Perfume                      Brand    Year  \
0                                   jana             verset-parfums  2020.0   
1                       ingenious-ginger  goldfield-banks-australia  2023.0   
2                            lily-splash                o-boticario  2021.0   
3                               imperium                 electimuss  2020.0   
4                                 torero    boadicea-the-victorious  2021.0   
5                           eudora-royal                     eudora  2022.0   
6  reckless-pour-femme-essence-de-parfum                  roja-dove  2021.0   
7                        essencial-unico                     natura  2022.0   
8           